# Launch LBANN traing jobs
Code to create a batch script for launching jobs on cori GPU

Sep 1, 2020

In [1]:
import os
import glob,time
import subprocess as sp
import numpy as np

import yaml


In [2]:
from ipywidgets import *

In [3]:
start_dir=os.getcwd()
print(start_dir)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn


## Define dictionary

In [39]:
## Read ref_launch.yaml
# main_code_dir=config_dict[facility]['main_code_dir']
# dict_pars=config_dict[facility][img_size][run_type]

### 64cube
dict_pars={'nodes':10, 'queue':'regular','bins':'unenven','time':'0:30:00',
           'job_name':'3ptfnc',
            'suffix':'inf_20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50_step_42-24720_sigma0.8','num_l':8,'img_slice':64,
                 }

### 128cube
dict_pars={'nodes':30, 'queue':'regular','bins':'unenven','time':'6:00:00',
           'job_name':'3ptfnc',
            'suffix':'val_3d_128cube_sigma0.5','num_l':8,'img_slice':128,
                 }

dict_pars['staging_loc']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn/'
dict_pars['code_dir']='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn'
dict_pars['cores']=dict_pars['nodes']*32


In [40]:
### 3dgan 64cube validation
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'
# dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210506_175558_64cube_bs8_lr0.0006_nodes8_spec0.1_bestrun/images/gen_img_epoch-124_step-19390.npy'

In [41]:
### 3dcgan 64cube
sigma=0.5
#dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma)
dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210617_204752_cgan-64_bs32_nodes1_lr0.0001-vary_fm50_goodrun/images/inference_label-{0}_epoch-42_step-24720.npy'.format(sigma)

In [42]:
### 3dgan 128cube validation
sigma=0.5
dict_pars['fname']='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_{0}_train_val.npy'.format(sigma)



In [43]:
# %%bash -s "$op_fldr"
# ls $1

In [44]:
dict_pars

{'nodes': 30,
 'queue': 'regular',
 'bins': 'unenven',
 'time': '6:00:00',
 'job_name': '3ptfnc',
 'suffix': 'val_3d_128cube_sigma0.5',
 'num_l': 8,
 'img_slice': 128,
 'staging_loc': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn/',
 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn',
 'cores': 960,
 'fname': '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy'}

In [45]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes={nodes}
#SBATCH --qos={queue}
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir={code_dir}
fname={fname}

srun -n {cores} python $code_dir/3pt_fcn_simple.py -n {num_l} --img_slice {img_slice} -idx $I -f $fname -sfx {suffix}

echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [46]:
fname='batch_3ptcn.sh'
filename=dict_pars['staging_loc']+fname
with open (filename,'w') as f:
    f.write(bash_strg)
print(filename)

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn/batch_3ptcn.sh


In [47]:
## Move to staging locations in project space:
os.chdir(dict_pars['staging_loc'])

In [48]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=30
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --account=m3363
#SBATCH -C haswell
#SBATCH --time=6:00:00
#SBATCH --job-name=3ptfnc

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

I=$1
source /global/common/software/m3035/conda-activate.sh 3.7
code_dir=/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_threeptfcn/run_3ptfcn
fname=/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset4_smoothing_4univ_cgan_varying_sigma_128cube/norm_1_sig_0.5_train_val.npy

srun -n 960 python $code_dir/3pt_fcn_simple.py -n 8 --img_slice 128 -idx $I -f $fname -sfx val_3d_128cube_sigma0.5

echo "--end date" `date` `date +%s`


## Submit job to cori GPU

In [49]:
lst=np.arange(32)
# lst=[0,3,6,8,19]
lst2=[str(i) for i in lst][0]
strg=' '.join(lst2)
print(strg)

0


In [50]:
%%bash -s "$filename" "$strg" ## Use python variable in bash
# for i in $2; do echo $i; echo $1; done
for i in $2; do sbatch $1 $i; done

Submitted batch job 43738685


In [15]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210617_204752_cgan_bs32_nodes8_lr0.0001-vary_fm50/images/inference_label-0.5_epoch-36_step-21550.npy'
# np.load(fname,mmap_mode='r').shape